In [1]:
from pre_nlp import featureset
import tensorflow as tf
import numpy as np

pos = 'C:/Users/chiay/Downloads/git/tensorflow/pos.txt'
neg = 'C:/Users/chiay/Downloads/git/tensorflow/neg.txt'
xtrain,ytrain,xtest,ytest = featureset(pos,neg,test_fraction=0.1)

C:\Python35\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


lexicon length = 423
total sample size = 10662
train = 9596, test = 1066


In [2]:
# set up three hidden layers, each with 1500 nodes
nodes1 = 1500
nodes2 = 1500
nodes3 = 1500
# number of classes, also is the number of nodes in output layer
n_classes = 2
# process data in batches given computing constraints
batch_size = 100
# x and y data variables
x = tf.placeholder('float')
y = tf.placeholder('float')

In [3]:
def neural_network_model(data):
    # laying out structure of neural network, no computation formulas yet
    # weights are matrices with shape m*n
    # m is number of input nodes, n is number of output nodes for that layer
    # biases are vectors with length l
    # l is number of output nodes for that layer
    # here the weights and biases are initialised randomly
    def layer(i,o):
        return {'weights':tf.Variable(tf.random_normal([i,o])),
               'biases': tf.Variable(tf.random_normal([o]))}
    
    hl1 = layer(len(xtrain[0]),nodes1)
    hl2 = layer(nodes1, nodes2)
    hl3 = layer(nodes2, nodes3)
    ol = layer(nodes3, n_classes)

    # now defining the mathematical flow of data through the neural network
    # at each layer, output activation value = relu((input x weights) + biases)
    def activate(i,o):
        value = tf.add(tf.matmul(i, o['weights']), o['biases'])
        return tf.nn.relu(value)
    
    l1 = activate(data, hl1)
    l2 = activate(l1, hl2)
    l3 = activate(l2, hl3)
    output = tf.matmul(l3,ol['weights']) + ol['biases']

    return output

In [4]:
def train_neural_network(x):
    prediction = neural_network_model(x)
    # cost function
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
    # minimise cost with Adam Optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
    # each epoch is a training iteration where we calculate the cost and optimize weights and biases
    epochs = 10
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        # for each epoch:
        for epoch in range(epochs):
            epoch_loss = 0
            i = 0
            # for each batch:
            while i < len(xtrain):
                start = i 
                end = i + batch_size
                xbatch = np.array(xtrain[start:end])
                ybatch = np.array(ytrain[start:end])
                n, c = sess.run([optimizer, cost], feed_dict={x: xbatch, y: ybatch})
                epoch_loss += c
                i += batch_size

            print('Epoch', epoch + 1, 'completed out of',epochs,'loss:',epoch_loss)
        # tf.argmax gives index (label) of the highest value (confidence) across the axis 
        # basically turns the matrixes of probabilities into a vector of predicted label values
        # tf.argmax tells us if the predicted label matches the actual label
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        #calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:xtest, y:ytest}))

train_neural_network(x)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Epoch 1 completed out of 10 loss: 1120109.0952148438
Epoch 2 completed out of 10 loss: 472558.9139404297
Epoch 3 completed out of 10 loss: 310383.08111572266
Epoch 4 completed out of 10 loss: 218865.90446472168
Epoch 5 completed out of 10 loss: 192007.9891052246
Epoch 6 completed out of 10 loss: 130097.50730514526
Epoch 7 completed out of 10 loss: 61213.93138885498
Epoch 8 completed out of 10 loss: 18224.823096990585
Epoch 9 completed out of 10 loss: 13066.511146306992
Epoch 10 completed out of 10 loss: 9540.859710216522
Accuracy: 0.6210131
